In [1]:
import ast
import inspect
import textwrap
from functools import wraps
from tqdm import tqdm


class TqdmTransformer(ast.NodeTransformer):
    """AST transformer that wraps all for loop iterables with tqdm()"""
    
    def visit_For(self, node):
        # First, recursively visit child nodes
        self.generic_visit(node)
        
        # Wrap the iterator with tqdm()
        # Create: tqdm(original_iter, desc="Loop description")
        new_iter = ast.Call(
            func=ast.Name(id='tqdm', ctx=ast.Load()),
            args=[node.iter],
            keywords=[
                ast.keyword(
                    arg='desc',
                    value=ast.Constant(value=f"Loop at line {node.lineno}")
                )
            ]
        )
        
        # Replace the iterator
        node.iter = new_iter
        
        return node


def auto_tqdm(func):
    """
    Decorator that automatically wraps all for loops in a function with tqdm.
    
    Example:
        @auto_tqdm
        def process_data(items):
            results = []
            for item in items:
                results.append(item * 2)
            return results
    """
    @wraps(func)
    def wrapper(*args, **kwargs):
        # Get the source code of the function
        source = inspect.getsource(func)
        
        # Remove the decorator line(s) to avoid recursion
        source_lines = source.split('\n')
        # Find where the actual function definition starts
        func_start = next(i for i, line in enumerate(source_lines) 
                         if line.strip().startswith('def '))
        source = '\n'.join(source_lines[func_start:])
        
        # Dedent the source code
        source = textwrap.dedent(source)
        
        # Parse the source into an AST
        tree = ast.parse(source)
        
        # Transform the AST to add tqdm to for loops
        transformer = TqdmTransformer()
        new_tree = transformer.visit(tree)
        
        # Fix missing locations in the AST
        ast.fix_missing_locations(new_tree)
        
        # Compile the modified AST
        code = compile(new_tree, filename='<ast>', mode='exec')
        
        # Create a new namespace with tqdm available
        namespace = {'tqdm': tqdm}
        # Add the function's globals to the namespace
        namespace.update(func.__globals__)
        
        # Execute the modified code in the namespace
        exec(code, namespace)
        
        # Get the modified function
        modified_func = namespace[func.__name__]
        
        # Call the modified function
        return modified_func(*args, **kwargs)
    
    return wrapper


# Example usage
if __name__ == "__main__":
    @auto_tqdm
    def example_function(n):
        """Function with multiple for loops"""
        results = []
        
        # First loop
        for i in range(n):
            # Nested loop
            for j in range(100):
                results.append(i * j)
        
        # Another separate loop
        for item in ['a', 'b', 'c']:
            results.append(item)
        
        return results
    
    # Run the function - all loops will have progress bars!
    output = example_function(50)
    print(f"\nProcessed {len(output)} items")

Loop at line 12: 100%|██████████| 3/3 [00:00<00:00, 54471.48it/s]


Processed 5003 items
